In [ ]:
import re

import jsonlines
import pandas as pd
import numpy as np

In [2]:
# # TODO: load metadata and return aggregate / summary statistics
# # TODO: write method to display cover image w/ cover metadata and add annotations to image
# # TODO: consider ideate/innotater for annotating directly in Jupyter notebooks

logged_metadata = []

with jsonlines.open('../metadata/covers.jsonl', mode='r') as reader:
    for item in reader:
        logged_metadata.append(item)
        
df = pd.DataFrame(logged_metadata)

In [3]:
def get_issue_number_from_title(title):
    issue = re.search(r"([#?])(\d+)\b", title.replace(',', ''))
    if issue is None:
        return np.nan
    else:
        return np.int(issue.group().replace('#', ''))

df['issue_number'] = df['title'].apply(get_issue_number_from_title)

df.drop(['issue_number', 'covers'], axis=1).describe().T

,count,unique,top,freq
format_binding,15025,43,Saddle-stitched,6145
format_color,15025,15,Color,8617
format_dimensions,15025,49,Standard Modern Age US,2946
format_paper_stock,15025,52,,4379
format_publishing_format,15025,36,was ongoing series,5669
indexer_notes,15025,10034,This issue has variants. |,1911
indicia_frequency,15025,374,monthly,8121
issue_brand,15025,104,DC [bullet],2697
issue_indicia_publisher,15025,57,DC Comics,3774
issue_pages,15025,83,36,10499


In [4]:
df_covers = pd.concat(df['covers'].apply(lambda x: pd.DataFrame(x).T).tolist(), axis=0).reset_index(drop=True)

df_covers.tail()

,cover_characters,cover_colors,cover_editing,cover_first line of dialogue or text,cover_genre,cover_inks,cover_job number,cover_keywords,cover_letters,cover_pencils,cover_script,cover_synopsis,image_url,save_to
19251,Superman [Clark Kent; Kal-El]; Contessa Erica ...,Brett Breeding; Android Images (separations),NaN,NaN,superhero,Brett Breeding (signed),NaN,NaN,?,Tom Grummett (signed),NaN,NaN,https://files1.comics.org//img/gcd/covers_by_i...,./covers/Superman: The Man of Tomorrow: Superm...
19252,Superman [Clark Kent; Kal-El]; Lex Luthor,Brett Breeding; Android Images (separations),NaN,NaN,superhero,Brett Breeding (signed),NaN,NaN,?,Tom Grummett (signed),NaN,NaN,https://files1.comics.org//img/gcd/covers_by_i...,./covers/Superman: The Man of Tomorrow: Superm...
19253,Superman [Clark Kent; Kal-El]; Captain Marvel ...,Color Works,NaN,NaN,superhero,Brett Breeding (signed),NaN,NaN,?,Tom Grummett (signed),NaN,NaN,https://files1.comics.org//img/gcd/covers_by_i...,./covers/Superman: The Man of Tomorrow: Superm...
19254,Superman [Clark Kent; Kal-El]; Lex Luthor; Con...,Patrick Martin,NaN,NaN,superhero,Brett Breeding (signed),NaN,NaN,?,Tom Grummett (signed),NaN,NaN,https://files1.comics.org//img/gcd/covers_by_i...,./covers/Superman: The Man of Tomorrow: Superm...
19255,Superman; Jackal,Patrick Martin,NaN,NaN,superhero,Brett Breeding [as Breeding] (signed),NaN,NaN,NaN,Tom Grummett [as Grummet] (signed),NaN,NaN,https://files1.comics.org//img/gcd/covers_by_i...,./covers/Superman: The Man of Tomorrow: Superm...


In [5]:
flatten = lambda l: [item for sublist in l for item in sublist]

In [6]:
characters = list(df_covers["cover_characters"].dropna().values)
print(len(characters))

16044


In [19]:
test = characters[12000:12050]
test[1]

'Generation X [Emma Frost [White Queen]; M; Husk; Chamber; Jubilee]; Gaia; Dark Beast'

In [81]:
import difflib

def check_if_team(characters: str):
    left_bracket_counter = 0
    is_team = False
    for idx, val in enumerate(characters):
        if val == "[" and (left_bracket_counter == 0):
            left_bracket_counter += 1
            continue
        if (val == "]") and (left_bracket_counter > 0):
            left_bracket_counter -= 1
            continue
        if val == "[" and (left_bracket_counter >= 1):
            is_team = True
            continue
    return is_team

def replace_semicolons_in_brackets(characters: str):
    p = re.compile(r"\[(.*?)\]")
    matches = []
    for m in p.finditer(characters):
        matches.append((m.start(), m.end(), m.group()))
    for match in matches:
        substring = characters[match[0]: match[1]]
        characters = characters.replace(substring,  substring.replace("; ", " | "))
    return characters

def parse_team_of_characters(characters: str):
    left_bracket_counter = 0
    for idx, val in enumerate(characters):
        if val == "[" and (left_bracket_counter == 0):
            k_idx = idx
            left_bracket_counter += 1
        if val == "[" and (left_bracket_counter > 0):
            left_bracket_counter += 1
        if (val == "]") and (left_bracket_counter > 0):
            left_bracket_counter -= 1
        if (val == "]") and (left_bracket_counter == 1):
            v_idx = idx
    k = characters[:k_idx]
    v = characters[k_idx+1:v_idx]
    return {k.strip(): v.strip()}

def parse_characters(characters:  str):
    print("characters:", characters)
    is_team = check_if_team(characters)
    print("is_team", is_team)
    if not is_team:
        res = [x.strip() for x in replace_semicolons_in_brackets(characters).split(";")]
        print("parsed characters:", parsed_characters)
    else:
        parsed_characters_dict = parse_team_of_characters(characters)
        print("parsed  characters dict:", parsed_characters_dict)
        for k in parsed_characters_dict:
            is_team = check_if_team(parsed_characters_dict[k])
            if not is_team:
                team = [x.strip() for x in replace_semicolons_in_brackets(parsed_characters_dict[k]).split(";")]
                res = {k:  team}
                full_res = str(res).replace("{", "").replace("}", "").replace("'", "").replace(",", ";")
                s = ""
                for idx, val in enumerate(difflib.ndiff(full_res, characters)):
                    if val[0] == "+":
                        s += val[2]
                non_team = list(filter(lambda x: x != "", s.split("; ")))
                res["Independent"] = non_team
    return res
        
    
res = parse_characters(test[0])

characters: Generation X [Chamber; Synch; Skin; Maggott]; Slaughter
is_team False
parsed characters: Generation X: [Emma Frost [White Queen]; M; Husk; Chamber; Jubilee]


In [82]:
res

['Generation X [Chamber | Synch | Skin | Maggott]', 'Slaughter']

In [9]:
# get list of unique characters across all  covers
def get_value_counts(df, column):
    """
    """
    return pd.Series("; ".join(df[column].\
                               dropna()).\
                     replace(" (signed)", "").\
                     replace(" (painted)", "").\
                     split('; ')).value_counts()

get_value_counts(df_covers, 'cover_characters')[:30]

Superman                                        1195
Batman [Bruce Wayne]                            1083
Iron Man [Tony Stark]                            925
Spider-Man [Peter Parker]                        887
Batman                                           690
Hulk [Bruce Banner]                              589
Kal-El]                                          572
Superman [Clark Kent                             532
Conan                                            515
Human Torch [Johnny Storm]                       423
Wolverine                                        373
Superman [Clark Kent]                            329
Cyclops                                          293
Captain America [Steve Rogers]                   282
Superboy                                         269
Lois Lane                                        267
Robin [Dick Grayson]                             260
Beast                                            239
Storm                                         

In [10]:
get_value_counts(df_covers, 'cover_pencils')[:20]

Gil Kane            409
Curt Swan           408
Jack Kirby          376
John Byrne          292
John Buscema        292
?                   276
Alex Ross           260
John Romita         229
Neal Adams          181
Herb Trimpe         173
Nick Cardy          173
Rich Buckler        167
George Pérez        157
Paul Ryan           151
Ryan Ottley         123
Win Mortimer        123
Jim Aparo           123
Greg Capullo        120
Tom Grummett        116
Salvador Larroca    116
dtype: int64

In [11]:
get_value_counts(df,  "series_name")[:20]

Action Comics              946
Batman                     865
Superman                   817
The Amazing Spider-Man     733
Fantastic Four             577
Iron Man                   451
The Avengers               409
The Incredible Hulk        407
Superboy                   401
Conan the Barbarian        293
Aquaman                    267
Strange Adventures         252
X-Factor                   251
Supergirl                  247
Avengers                   242
X-Men                      226
Batgirl                    172
The X-Men                  165
X-Force                    163
Guardians of the Galaxy    148
dtype: int64

In [12]:
get_value_counts(df_covers,  "cover_genre")[:20]

superhero                        16247
science fiction                    661
sword and sorcery                  640
humor                              316
fantasy-supernatural               309
anthropomorphic-funny animals      143
horror-suspense                    134
children                            98
adventure                           96
spy                                 83
crime                               82
satire-parody                       78
martial arts                        76
romance                             52
war                                 51
jungle                              47
western-frontier                    43
teen                                35
historical                          29
medical                             14
dtype: int64

In [ ]:
def create_training_dirs():
    """
    Given some args, create two compressed files: images.tar.gz & annos.tar.gz
    """
    
    

In [ ]:
#TODO: make this work (e.g. pull in series to df_covers... probably just all issue metadata honestly...)
def concat_values_by_series(df, concat_column, series_name):    

    list_of_synopsis = df[
        (df['series_name'] == series_name) & df['issue_number'] != 0.0].sort_values('issue_number')[concat_column]

    return " | ".join(list_of_synopsis.values)

print(concat_values_by_series(df, concat_column='synopsis', series_name="Detective Comics")[:8000], '...')